In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_3D(a, mu, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    while np.linalg.norm(x, 2) < a(rt):
        x += mu*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(x[2], x[1])   
    
    return rt+ndt, (theta1, theta2)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_3D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    mu = np.array([prms[2], prms[3], prms[4]])
    
    fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.0001, 
                        dt=0.05, T_max=max(RT))
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001 and ndt >= 0:
            mu_dot_x0 = mu[0]*np.cos(theta[0])
            mu_dot_x1 = mu[1]*np.sin(theta[0])*np.cos(theta[1]) 
            mu_dot_x2 = mu[2]*np.sin(theta[0])*np.sin(theta[1])
            term1 = prms[0] * (mu_dot_x0 + mu_dot_x1 + mu_dot_x2)
            term2 = 0.5 * np.linalg.norm(mu, 2)**2 * (rt-ndt)
            
            density = np.exp(term1 - term2) * fpt(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
recovery_df = {'threshold_true': [],
               'threshold_estimate': [],
               'ndt_true': [],
               'ndt_estimate': [],
               'mu1_true': [],
               'mu1_estimate': [],
               'mu2_true': [],
               'mu2_estimate': [],
               'mu3_true': [],
               'mu3_estimate': []}

min_threshold = 0.5
max_threshold = 6

min_ndt = 0.1
max_ndt = 1

min_mu = -6
max_mu = 6

In [6]:
for n in tqdm(range(50)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(50):
        rt, theta = simulate_HSDM_3D(a, mu, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_3D_likelihood,
                                     args=(RT, Theta), 
                                     bounds=[(min_threshold, max_threshold), (min_ndt, max_ndt),
                                             (min_mu, max_mu), (min_mu, max_mu), (min_mu, max_mu)])
    
    recovery_df['threshold_true'].append(threshold)
    recovery_df['ndt_true'].append(ndt)
    recovery_df['mu1_true'].append(mu[0])
    recovery_df['mu2_true'].append(mu[1])
    recovery_df['mu3_true'].append(mu[2])
    
    recovery_df['threshold_estimate'].append(min_ans.x[0])
    recovery_df['ndt_estimate'].append(min_ans.x[1])
    recovery_df['mu1_estimate'].append(min_ans.x[2])
    recovery_df['mu2_estimate'].append(min_ans.x[3])
    recovery_df['mu3_estimate'].append(min_ans.x[4])
    

100%|███████████████████████████████████| 50/50 [18:53<00:00, 22.67s/it]


In [7]:
recovery_df = pd.DataFrame(recovery_df)
recovery_df.corr()

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
threshold_true,1.000000,0.974019,0.106514,0.107337,-0.036430,-0.020309,-0.314513,-0.318861,-0.065492,-0.108204
threshold_estimate,0.974019,1.000000,0.146036,0.102642,-0.049798,-0.032507,-0.327403,-0.333022,-0.018020,-0.058923
ndt_true,0.106514,0.146036,1.000000,0.961601,-0.047240,-0.030866,-0.054457,-0.083050,0.004404,0.016142
ndt_estimate,0.107337,0.102642,0.961601,1.000000,-0.073646,-0.055734,-0.064150,-0.089775,-0.021117,-0.007062
mu1_true,-0.036430,-0.049798,-0.047240,-0.073646,1.000000,0.995681,0.004969,0.006491,0.015811,-0.011036
mu1_estimate,-0.020309,-0.032507,-0.030866,-0.055734,0.995681,1.000000,0.023650,0.023956,-0.005385,-0.030685
mu2_true,-0.314513,-0.327403,-0.054457,-0.064150,0.004969,0.023650,1.000000,0.996568,-0.084547,-0.057996
mu2_estimate,-0.318861,-0.333022,-0.083050,-0.089775,0.006491,0.023956,0.996568,1.000000,-0.080063,-0.052871
mu3_true,-0.065492,-0.018020,0.004404,-0.021117,0.015811,-0.005385,-0.084547,-0.080063,1.000000,0.992931
mu3_estimate,-0.108204,-0.058923,0.016142,-0.007062,-0.011036,-0.030685,-0.057996,-0.052871,0.992931,1.000000


In [8]:
file_name = '_Recovery_data/IE_3d_recovery_50_05.csv'
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, 
                         recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df

,threshold_true,threshold_estimate,ndt_true,ndt_estimate,mu1_true,mu1_estimate,mu2_true,mu2_estimate,mu3_true,mu3_estimate
0,2.399754,3.083665,0.952112,0.891003,-5.888421,-6.000000,-1.243302,-1.599813,4.595188,4.703245
1,1.140770,1.135422,0.775084,0.785419,-2.673946,-2.493159,0.182034,0.553383,-4.980074,-4.976956
2,3.484301,3.245300,0.876095,0.986661,1.388852,1.531190,-2.068111,-2.487549,3.259216,3.494372
3,5.037450,5.484156,0.852365,0.787544,-1.029203,-1.044037,-0.224508,-0.331033,3.851466,4.022561
4,1.538695,1.598135,0.892056,0.872056,1.978392,2.116671,2.253707,1.949094,1.967265,2.460060
...,...,...,...,...,...,...,...,...,...,...
295,2.647721,2.962630,0.754005,0.775337,-4.868429,-6.000000,-1.691368,-1.974778,3.945340,5.221824
296,5.236120,5.503403,0.260567,0.323798,-0.231332,-0.183637,4.361222,4.938161,4.745049,5.252349
297,5.681297,6.000000,0.686279,0.723973,5.433786,5.727224,-3.432367,-3.707569,0.614321,0.731593
298,2.813896,2.295852,0.101485,0.189404,5.958701,6.000000,3.683197,3.882698,-0.933589,-1.144453
